# LLM Iteration 5

**Aim**

Change length of stay time depending on patient and round the output of waiting times and admission and discharge.

**Prompt**

Update this model to change the length of stay depending on the patient type. The length of stay distribution and parameters of patients are dependent on the patient type. For each distribution time is measured in days. 
- Stroke = Lognormal: mean 7.4, standard deviation 8.6
- TIA = Lognormal: mean 1.8, standard deviation 2.3
- Complex Neurological = Lognormal: mean 4.0, standard deviation 5.0
- Other = Lognormal: mean 3.8, standard deviation 5.2

The inputs for the lognormal distribution are mu and sigma and should be calculated as follows, where v is the standard deviation squared and m is the mean:
- phi = math.sqrt(v + m^2)
- mu = math.log(m^2/phi)
- sigma = math.sqrt(math.log(phi^2/m^2))

Numpy should be used for sampling. Each length of stay distribution should have its own numpy.random.Generator object. Each random number generator should use the random seed specified for each patient type. 

Round the time at which patients are admitted and discharged and the wait times in the model output.

**Note**

Modifications have been made to the output from the LLM, indicated by:
```python
########## MODIFICATION ##########
change
##################################
```
This is either used to add in code from previous iterations if only certain sections were updated, used to correct the LLM's code if required, or used to check the model is working as expected.

**LLM Output**

In [1]:
import simpy
import numpy as np
import math

In [2]:
class StrokeWard:
    def __init__(self, env, num_beds):
        self.env = env
        self.beds = simpy.Resource(env, capacity=num_beds)
        self.patient_count = 0
        self.wait_times = []
        self.length_of_stay = {
            "Stroke": {"mu": 7.4, "sigma": 8.6},
            "TIA": {"mu": 1.8, "sigma": 2.3},
            "Complex Neurological": {"mu": 4.0, "sigma": 5.0},
            "Other": {"mu": 3.8, "sigma": 5.2}
        }

        ########## MODIFICATION ##########
        self.patients_in_beds = 0
        self.bed_in_use_store = []
        self.los_store = []
        self.patient_type_store = []
        ##################################

    def admit_patient(self, patient):
        arrival_time = round(self.env.now)
        with self.beds.request() as bed:
            yield bed
            ########## MODIFICATION ##########
            bed_arrival_time = self.env.now
            self.patients_in_beds += 1
            self.bed_in_use_store.append(self.patients_in_beds)
            self.patient_type_store.append(patient.patient_type)
            ##################################

            wait_time = round(self.env.now) - arrival_time
            self.wait_times.append(wait_time)
            print(f'Patient {patient.id} ({patient.patient_type}) admitted to '
                  f'the stroke ward at day {round(self.env.now)}. '
                  f'Wait time: {wait_time} days')
            los = np.random.default_rng(patient.seed).\
                lognormal(self.length_of_stay[patient.patient_type]["mu"], 
                          self.length_of_stay[patient.patient_type]["sigma"])
            # Length of stay based on patient type
            yield self.env.timeout(round(los))
            ########## MODIFICATION ##########
            self.patients_in_beds -= 1
            bed_leave_time = self.env.now
            self.los_store.append(bed_leave_time - bed_arrival_time)
            ##################################
            print(f'Patient {patient.id} ({patient.patient_type}) discharged '
                  f'from the stroke ward at day {round(self.env.now)}')

class Patient:
    def __init__(self, id, patient_type, seed):
        self.id = id
        self.patient_type = patient_type
        self.seed = seed

def stroke_arrival(env, stroke_ward, rng):
    while True:
        interarrival_time = rng.exponential(1.2)
        yield env.timeout(interarrival_time)
        patient = Patient(stroke_ward.patient_count, "Stroke",
                          rng.integers(0, np.iinfo(np.int32).max))
        stroke_ward.patient_count += 1
        env.process(stroke_ward.admit_patient(patient))

def tia_arrival(env, stroke_ward, rng):
    while True:
        interarrival_time = rng.exponential(9.3)
        yield env.timeout(interarrival_time)
        patient = Patient(stroke_ward.patient_count, "TIA",
                          rng.integers(0, np.iinfo(np.int32).max))
        stroke_ward.patient_count += 1
        env.process(stroke_ward.admit_patient(patient))

def complex_neurological_arrival(env, stroke_ward, rng):
    while True:
        interarrival_time = rng.exponential(3.6)
        yield env.timeout(interarrival_time)
        patient = Patient(stroke_ward.patient_count, "Complex Neurological",
                          rng.integers(0, np.iinfo(np.int32).max))
        stroke_ward.patient_count += 1
        env.process(stroke_ward.admit_patient(patient))

def other_arrival(env, stroke_ward, rng):
    while True:
        interarrival_time = rng.exponential(3.2)
        yield env.timeout(interarrival_time)
        patient = Patient(stroke_ward.patient_count, "Other",
                          rng.integers(0, np.iinfo(np.int32).max))
        stroke_ward.patient_count += 1
        env.process(stroke_ward.admit_patient(patient))

In [3]:
env = simpy.Environment()
stroke_ward = StrokeWard(env, num_beds=10)

# Set random seed for reproducibility for each patient type
random_seeds = {
    "Stroke": 12345,
    "TIA": 54321,
    "Complex Neurological": 98765,
    "Other": 67890
}

rngs = {patient_type: np.random.default_rng(seed) for patient_type, \
        seed in random_seeds.items()}

env.process(stroke_arrival(env, stroke_ward, rngs["Stroke"]))
env.process(tia_arrival(env, stroke_ward, rngs["TIA"]))
env.process(complex_neurological_arrival(env, stroke_ward,
                                         rngs["Complex Neurological"]))
env.process(other_arrival(env, stroke_ward, rngs["Other"]))

# Set the run length to 2 months (default)
run_length_days = 60 * 2

print("Simulation started...")
env.run(until=run_length_days)
print("Simulation ended.")

# Calculate and print the mean wait time for a bed
mean_wait_time = np.mean(stroke_ward.wait_times)
print(f"Mean wait time for a bed: {mean_wait_time} days")

Simulation started...
Patient 0 (Complex Neurological) admitted to the stroke ward at day 0. Wait time: 0 days
Patient 0 (Complex Neurological) discharged from the stroke ward at day 0
Patient 1 (Stroke) admitted to the stroke ward at day 0. Wait time: 0 days
Patient 2 (TIA) admitted to the stroke ward at day 2. Wait time: 0 days
Patient 3 (Complex Neurological) admitted to the stroke ward at day 4. Wait time: 0 days
Patient 4 (Other) admitted to the stroke ward at day 5. Wait time: 0 days
Patient 5 (Stroke) admitted to the stroke ward at day 6. Wait time: 0 days
Patient 6 (Stroke) admitted to the stroke ward at day 6. Wait time: 0 days
Patient 7 (Other) admitted to the stroke ward at day 7. Wait time: 0 days
Patient 2 (TIA) discharged from the stroke ward at day 7
Patient 8 (Complex Neurological) admitted to the stroke ward at day 8. Wait time: 0 days
Patient 9 (Stroke) admitted to the stroke ward at day 8. Wait time: 0 days
Patient 10 (Other) admitted to the stroke ward at day 9. Wai

This updated Python code now incorporates different lengths of stay for each patient type in the simulation model of a stroke ward in a hospital using SimPy 4. The length of stay is calculated based on a lognormal distribution with specific mean and standard deviation values for each patient type. The model also includes rounding for admission and discharge times as well as wait times in the output.

**Checks**

In [4]:
class TestCase:
    def __init__(self, env, ward, rngs):
        self.env = env
        self.ward = ward
        env.process(stroke_arrival(env, ward, rngs["Stroke"]))
        env.process(tia_arrival(env, ward, rngs["TIA"]))
        env.process(complex_neurological_arrival(env, ward,
                                                 rngs["Complex Neurological"]))
        env.process(other_arrival(env, ward, rngs["Other"]))

    def test_run_length(self):
        '''
        Checks that specified run length is implemented correctly.
        '''
        self.env.run(until=RUN_LENGTH)

        if self.env.now > RUN_LENGTH:
            print('The simulation has continued for longer than input.')

    def test_bed_hold(self):
        '''
        Checks that patients are holding beds until they leave the model.
        '''
        self.env.run(until=RUN_LENGTH)

        for bed_use in self.ward.bed_in_use_store:
            if bed_use > NUM_BEDS:
                print('The number of beds in use exceeded limit.')

    def test_length_of_stay(self):
        '''
        Prints out patients' length of stay to check they are reasonable.
        '''
        self.env.run(until=RUN_LENGTH)

        print('\nLength of stay:')
        for pat_type, los in zip(self.ward.patient_type_store,
                                 self.ward.los_store):
            print(f'({pat_type}) {los}')

In [5]:
# Checks run length
NUM_BEDS = 10
RUN_LENGTH = 10

random_seeds = {
    "Stroke": 12345,
    "TIA": 54321,
    "Complex Neurological": 98765,
    "Other": 67890
}

rngs = {patient_type: np.random.default_rng(seed) for patient_type, \
        seed in random_seeds.items()}

env = simpy.Environment()
ward = StrokeWard(env, NUM_BEDS)

test = TestCase(env, ward, rngs)
test.test_run_length()

Patient 0 (Complex Neurological) admitted to the stroke ward at day 0. Wait time: 0 days
Patient 0 (Complex Neurological) discharged from the stroke ward at day 0
Patient 1 (Stroke) admitted to the stroke ward at day 0. Wait time: 0 days
Patient 2 (TIA) admitted to the stroke ward at day 2. Wait time: 0 days
Patient 3 (Complex Neurological) admitted to the stroke ward at day 4. Wait time: 0 days
Patient 4 (Other) admitted to the stroke ward at day 5. Wait time: 0 days
Patient 5 (Stroke) admitted to the stroke ward at day 6. Wait time: 0 days
Patient 6 (Stroke) admitted to the stroke ward at day 6. Wait time: 0 days
Patient 7 (Other) admitted to the stroke ward at day 7. Wait time: 0 days
Patient 2 (TIA) discharged from the stroke ward at day 7
Patient 8 (Complex Neurological) admitted to the stroke ward at day 8. Wait time: 0 days
Patient 9 (Stroke) admitted to the stroke ward at day 8. Wait time: 0 days
Patient 10 (Other) admitted to the stroke ward at day 9. Wait time: 0 days
Patient

In [6]:
# Checks that is only ever 1 patient if there is only 1 bed
NUM_BEDS = 1
RUN_LENGTH = 60

random_seeds = {
    "Stroke": 12345,
    "TIA": 54321,
    "Complex Neurological": 98765,
    "Other": 67890
}

rngs = {patient_type: np.random.default_rng(seed) for patient_type, \
        seed in random_seeds.items()}

env = simpy.Environment()
ward = StrokeWard(env, NUM_BEDS)

test = TestCase(env, ward, rngs)
test.test_bed_hold()

Patient 0 (Complex Neurological) admitted to the stroke ward at day 0. Wait time: 0 days
Patient 0 (Complex Neurological) discharged from the stroke ward at day 0
Patient 1 (Stroke) admitted to the stroke ward at day 0. Wait time: 0 days


In [7]:
# Checks length of stay for patients
NUM_BEDS = 10
RUN_LENGTH = 60

random_seeds = {
    "Stroke": 12345,
    "TIA": 54321,
    "Complex Neurological": 98765,
    "Other": 67890
}

rngs = {patient_type: np.random.default_rng(seed) for patient_type, \
        seed in random_seeds.items()}

env = simpy.Environment()
ward = StrokeWard(env, NUM_BEDS)

test = TestCase(env, ward, rngs)
test.test_length_of_stay()

Patient 0 (Complex Neurological) admitted to the stroke ward at day 0. Wait time: 0 days
Patient 0 (Complex Neurological) discharged from the stroke ward at day 0
Patient 1 (Stroke) admitted to the stroke ward at day 0. Wait time: 0 days
Patient 2 (TIA) admitted to the stroke ward at day 2. Wait time: 0 days
Patient 3 (Complex Neurological) admitted to the stroke ward at day 4. Wait time: 0 days
Patient 4 (Other) admitted to the stroke ward at day 5. Wait time: 0 days
Patient 5 (Stroke) admitted to the stroke ward at day 6. Wait time: 0 days
Patient 6 (Stroke) admitted to the stroke ward at day 6. Wait time: 0 days
Patient 7 (Other) admitted to the stroke ward at day 7. Wait time: 0 days
Patient 2 (TIA) discharged from the stroke ward at day 7
Patient 8 (Complex Neurological) admitted to the stroke ward at day 8. Wait time: 0 days
Patient 9 (Stroke) admitted to the stroke ward at day 8. Wait time: 0 days
Patient 10 (Other) admitted to the stroke ward at day 9. Wait time: 0 days
Patient

**Reflection**

The LLM did not implement the lognormal distribution for length of stay correctly, as it assumed that the values passed were the mu and sigma values required, despite the equation being specified in the prompt. This meant the patients' length of stay were not representative of the true distribution and caused a lot of queuing in the model.